In [2]:
import pandas as pd
import numpy as np

import joblib

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
file_path = 'laptop_price.csv'
df = pd.read_csv(file_path)

# базовая инфо
print(
    df.shape,
    "Кусок считанных данных:",
    df.head(),
    sep="\n"
)
df.info()

# логарифмированная цена будет использоваться для тренировки
df['Price_Log'] = np.log1p(df['Price'])

# цена нам больше не нужна
X = df.drop(columns=['Price', 'Price_Log'])
y = df['Price_Log']

# разбиваем данные для тренировки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("==================================")

print(
    f"{X_train.shape=}",
    f"{X_test.shape=}",
    f"=== Кусок X_train: ===\n{X_train.head()}",
    f"=== Типы данных: ===\n{X_train.dtypes}",
    sep="\n"
)

(1000, 7)
Кусок считанных данных:
    Brand  Processor_Speed  RAM_Size  Storage_Capacity  Screen_Size    Weight  \
0    Asus         3.830296        16               512    11.185147  2.641094   
1    Acer         2.912833         4              1000    11.311372  3.260012   
2  Lenovo         3.241627         4               256    11.853023  2.029061   
3    Acer         3.806248        16               512    12.280360  4.573865   
4    Acer         3.268097        32              1000    14.990877  4.193472   

          Price  
0  17395.093065  
1  31607.605919  
2   9291.023542  
3  17436.728334  
4  32917.990718  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             1000 non-null   object 
 1   Processor_Speed   1000 non-null   float64
 2   RAM_Size          1000 non-null   int64  
 3   Storage_Capacity  1000 non

In [ ]:
# Определяем признаки в X_train
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist() # числовые
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()      # категориальные

print("(Числовые признаки):", numerical_features)
print("(Категориальные признаки):", categorical_features)

(Числовые признаки): ['Processor_Speed', 'RAM_Size', 'Storage_Capacity', 'Screen_Size', 'Weight']
(Категориальные признаки): ['Brand']


In [ ]:
# Пайплайн для числовых категорий
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Пайплайн для категориальных категорий
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Трансформер для применения и группировки признаков в один
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # на всякий случай
)

In [27]:
# испольузем модель регрессии со стандартными параметрами
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, objective='reg:squarederror')

# создаем полноценный пайплайн для машинного обучения
# сначала препроцессор, потом регрессор
ml_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])

# Обучение конвейера на тренировочных данных
print("Обучение...")
ml_pipeline.fit(X_train, y_train) # обучаем на предыдущих данных
print("завершено.")

# пиклим пайплайн в файл, чтобы его можно было сохранить
pipeline_filename = 'laptop_price_pipeline.pkl'
joblib.dump(ml_pipeline, pipeline_filename)
print(f"Пайплайн сохранен в {pipeline_filename}")

Обучение...
завершено.
Пайплайн сохранен в laptop_price_pipeline.pkl


In [ ]:
# теперь можно использовать модель для предсказания данных
y_pred_log = ml_pipeline.predict(X_test)

# возвращаемся от логарифмических цен к нормальным
y_pred_actual = np.expm1(y_pred_log)

# ditto
y_test_actual = np.expm1(y_test)

# Отображение нескольких примеров предсказаний по сравнению с фактическими значениями
comparison_df = pd.DataFrame({'Реальная цена': y_test_actual, 'Предсказ. цена': y_pred_actual})
print("Результаты тренировки")
print(comparison_df.head())

# Ура!

Результаты тренировки
     Реальная цена  Предсказ. цена
521   10900.502075    10523.270508
737   31727.516697    31459.175781
740    9636.022750     9646.339844
660    9412.599857     9260.269531
411   31641.105861    31977.861328
